In [1]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\leoni\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
seed = 42

In [5]:
import pandas as pd

from sklearn import preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
# from keras.preprocessing import text, sequence
# from keras import layers, models, optimizers



import pandas as pd
import numpy as np
import string
import re

from sklearn.base import BaseEstimator, TransformerMixin

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag, word_tokenize

stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [4]:
df = pd.read_csv("../data/tech_test_data-1.csv").drop("message_id", axis = 1)

customer_df = df[df["message_source"] == "customer"][["message","case_type"]]


## 2. Pre-processing Pipelines

#### 2.1 Pre-Processing methods

In [15]:
def lemmatize(phraze):
    new_phraze = []
    for word, tag in pos_tag(word_tokenize(phraze)):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            new_phraze.append(word)
        else:
            new_phraze.append(lemmatizer.lemmatize(word, wntag))
        
    return " ".join(new_phraze)

def replace_num(phrase):
    return re.sub(" \d+", " _number_", phrase)

def replace_orderID(phrase):
    return re.sub("([A-Za-z]+[\d@]+[\w@]*|[\d@]+[A-Za-z]+[\w@]*)", "_orderID_", phrase)

def remove_punc(phraze):
    return re.sub(r'[^\w\s]',"",phraze)

def remove_extra_space(phraze):
    return re.sub(' +', ' ', phraze)
    
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"hasn\’t", "has not", phrase)
    phrase = re.sub(r"haven\’t", "has not", phrase)
    phrase = re.sub(r"\’d", " would", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

#### 2.2 NLP pipeline

In [22]:
class NLP_Preprocessor(BaseEstimator, TransformerMixin):

    def __init__(self, variables=None):
        pass

    def fit(self, X, y=None):
        # we need the fit statement to accomodate the sklearn pipeline
        return self

    def transform(self, X):
        X = X.copy()
        X['message'] = X['message'].apply(lambda x: x.lower()).\
                apply(lambda x: decontracted(x)).\
                apply(lambda x: " ".join([item for item in x.split() if item not in stop_words])).\
                apply(lambda x: replace_num(x)).\
                apply(lambda x: replace_orderID(x)).\
                apply(lambda x: lemmatize(x)).\
                apply(lambda x: remove_punc(x)).\
                apply(lambda x: remove_extra_space(x))

        return X

In [23]:
nlp_processor = NLP_Preprocessor()
cleaned_df = nlp_processor.fit_transform(customer_df)

## Split dataset

In [19]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.4, random_state=seed)

nlp_processor = NLP_Preprocessor()
cleaned_df = nlp_processor.fit_transform(customer_df)

X = cleaned_df.drop('case_type', axis=1)
y = cleaned_df['case_type'].apply(lambda x: 0 if x=="cancel_order" else 1)


for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# X_train, X_test, y_train, y_test = train_test_split(X['message'], y)

X_train = X_train["message"]
X_test = X_test["message"]